To Do:
-Cycle through directory to process all zip files
-Process all files in zip file

-Load table of what items are associated with what items, trim table to only items created after 2015
-Determine where data stands
-Calculate cost per packet, divide cost per packet by number of items
-Scope out messy/unmatched data

In [70]:
import pandas as pd
import numpy
import xlrd
from zipfile import ZipFile

unzipped_datafile = "SCI_Beck, Kim_10-1-15.xlsx"
timesheet = pd.DataFrame([],columns=['Name', 'Invoice Date','Payrate','Hours','Activity','Description'])


In [71]:
def process_timesheet(unzipped_datafile, timesheet):
    workbook = xlrd.open_workbook(unzipped_datafile)
    sheet = workbook.sheet_by_index(0)

    #Process first tab of timesheet
    #On the first tab, save cell B3 as the contractor name
    name = sheet.cell_value(2, 1)
    #On the first tab, save cell D18 as the contractor rate
    payrate = sheet.cell_value(17,3)
    date = str(int(sheet.cell_value(2,4))) + "-" + str(int(sheet.cell_value(2,5))) + "-" + str(int(sheet.cell_value(2,6)))

    #Process second tab of Excel spreadsheet
    sheet = workbook.sheet_by_index(1)
    #On the second tab, starting with cell A6:E6, save each row until you encounter an empty row
    row = 5
    hours = sheet.cell_value(row,0)
    activity = sheet.cell_value(row, 1)
    description = sheet.cell_value(row, 4)
    while hours != '':
        timesheet = timesheet.append({'Name':name, 'Payrate':payrate, 'Invoice Date':date, 'Hours':hours, 'Activity':activity, 'Description':description}, ignore_index=True)
        row += 1
        hours = sheet.cell_value(row,0)
        activity = sheet.cell_value(row, 1)
        description = sheet.cell_value(row, 4)
    return timesheet

In [62]:
timesheet = process_timesheet(unzipped_datafile, timesheet)
    
print timesheet

            Name Invoice Date  Payrate  Hours          Activity  \
0  Kimberly Beck      10-1-15     30.0   4.00      Item Writing   
1  Kimberly Beck      10-1-15     30.0   5.75     Tagging - New   
2  Kimberly Beck      10-1-15     30.0   8.75  Tagging - Review   

                             Description  
0                        CR Items - Math  
1                     Math - Shapes, SIG  
2  Math - Numbers & digits, Units, Steps  


In [66]:
import numpy
import xlrd
from zipfile import ZipFile
datafile = "FAIB Contractor Invoices 4-16-16"


def open_zip(datafile):
    with ZipFile('{0}'.format(datafile), 'r') as myzip:
        myzip.extractall()        


In [68]:
#open_zip(datafile)

In [74]:
import glob

for file in glob.glob("*.zip"):    # only loops over zip files
        open_zip(file)
        
#apply process_timesheet to all unzipped Excel files
for file in glob.glob("*.xlsx"):
        timesheet = process_timesheet(file, timesheet)

In [75]:
print timesheet

                    Name Invoice Date Payrate  Hours                Activity  \
0           Greg Klinger      3-16-16      50  29.50            item writing   
1           Greg Klinger      3-16-16      50   8.00            item writing   
2           Greg Klinger      4-16-16      50  10.00            item writing   
3           Greg Klinger      4-16-16      50  10.00            item writing   
4           Greg Klinger      4-16-16      50  10.00            item writing   
5              Matt Akin      3-16-16      25   2.00            Item Writing   
6              Matt Akin      3-16-16      25   9.00            Item Writing   
7              Matt Akin      3-16-16      25   8.00            Item Writing   
8              Matt Akin      3-16-16      25   0.25           Miscellaneous   
9              Matt Akin      3-16-16      25   3.75            Item Writing   
10             Matt Akin      3-16-16      25   4.25            Item Writing   
11             Matt Akin      3-16-16   